# Chapter 19: Functions Deep Dive

---

## Introduction

Functions are the building blocks of JavaScript programming. While Chapter 5 introduced the basics of function syntax and usage, this chapter explores the advanced concepts that make JavaScript functions powerful, flexible, and central to modern development patterns. Understanding these concepts is essential for writing clean, maintainable, and efficient code.

In JavaScript, functions are **first-class citizens**, meaning they can be assigned to variables, passed as arguments, returned from other functions, and stored in data structures. This flexibility enables powerful programming paradigms like functional programming, callback patterns, and closure-based encapsulation.

By the end of this chapter, you will understand how to leverage advanced function patterns to write more modular, reusable, and robust JavaScript code that adheres to industry best practices.

---

## 19.1 Function Declarations vs Expressions

JavaScript provides two primary ways to define functions: declarations and expressions. While they may appear similar, they behave differently in crucial ways that affect how and when you can use them.

### Function Declarations

A function declaration defines a named function using the `function` keyword followed by the function name:

```javascript
// Function Declaration
function calculateArea(width, height) {
    return width * height;
}

// Usage
console.log(calculateArea(10, 5)); // 50
```

**Characteristics of Function Declarations:**

```javascript
// 1. HOISTING: Function declarations are hoisted to the top of their scope
// This means you can call the function BEFORE its declaration
console.log(greet("Alice")); // "Hello, Alice!"

function greet(name) {
    return `Hello, ${name}!`;
}

// 2. NAME PROPERTY: They have an internal name property
console.log(greet.name); // "greet"

// 3. BLOCK SCOPE: In strict mode, function declarations are block-scoped
// In non-strict mode, they leak to the containing function scope
function outer() {
    if (true) {
        function inner() {
            return "I am inner";
        }
        console.log(inner()); // Works here
    }
    // In strict mode: ReferenceError: inner is not defined
    // In non-strict mode: May work (implementation dependent)
    // console.log(inner());
}
```

### Function Expressions

A function expression defines a function as part of a larger expression syntax, typically by assigning it to a variable:

```javascript
// Function Expression (anonymous)
const calculateArea = function(width, height) {
    return width * height;
};

// Usage
console.log(calculateArea(10, 5)); // 50

// Named Function Expression
const factorial = function fact(n) {
    if (n <= 1) return 1;
    return n * fact(n - 1); // Can reference itself internally
};

console.log(factorial(5)); // 120
```

**Characteristics of Function Expressions:**

```javascript
// 1. NO HOISTING: Function expressions are NOT hoisted
// This will throw: TypeError: greet is not a function
// console.log(greet("Bob")); // ❌ Error

const greet = function(name) {
    return `Hello, ${name}!`;
};

// 2. ANONYMOUS vs NAMED: Can be anonymous or named
// Anonymous (common)
const add = function(a, b) {
    return a + b;
};

// Named (useful for recursion and debugging)
const fibonacci = function fib(n) {
    if (n <= 1) return n;
    return fib(n - 1) + fib(n - 2);
};

console.log(add.name);        // "add" (inferred from variable)
console.log(fibonacci.name);  // "fib" (explicit name)

// 3. MUST BE DEFINED BEFORE USE
const multiply = function(x, y) {
    return x * y;
};

console.log(multiply(4, 5)); // 20
```

### Key Differences Summary

```javascript
┌─────────────────────────────────────────────────────────────────┐
│          DECLARATION vs EXPRESSION COMPARISON                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Function Declaration              Function Expression          │
│  ────────────────────              ──────────────────           │
│                                                                 │
│  function sum(a, b) {              const sum = function(a, b) { │
│      return a + b;                     return a + b;            │
│  }                                 };                           │
│                                                                 │
│  • Hoisted to top of scope         • Not hoisted                │
│  • Can be called before            • Must be defined before use │
│    definition                                                   │
│  • Always has a name               • Can be anonymous           │
│  • Creates property on             • Does not create property   │
│    enclosing object (in non-        on enclosing object         │
│    strict mode)                                                 │
│  • Block-scoped in strict mode     • Always follows variable    │
│                                     scoping rules               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Industry Best Practices

**When to use Function Declarations:**
- When you need the function available throughout its scope (hoisting is beneficial)
- For top-level utility functions
- When readability of the function name at the top matters

**When to use Function Expressions:**
- When you want to control when the function is defined (conditional definition)
- For callbacks and event handlers
- When assigning methods to objects
- For IIFEs (Immediately Invoked Function Expressions)
- When the function should not be accessible before its definition (prevents temporal dead zone issues)

```javascript
// Best Practice: Use const with function expressions to prevent reassignment
const processData = function(data) {
    return data.map(item => item * 2);
};

// This will throw an error (good!)
// processData = "something else"; // TypeError: Assignment to constant variable

// Avoid var with function expressions (allows reassignment)
var badExample = function() {};
badExample = "oops"; // This works (bad!)
```

---

## 19.2 First-Class Functions

JavaScript treats functions as **first-class citizens** (or first-class objects), meaning functions are treated like any other value. This is one of JavaScript's most powerful features and enables functional programming patterns.

### What Are First-Class Functions?

```javascript
// In JavaScript, functions can:

// 1. Be assigned to variables
const sayHello = function() {
    console.log("Hello!");
};

// 2. Be stored in data structures (arrays, objects)
const functionArray = [
    function() { return 1; },
    function() { return 2; },
    function() { return 3; }
];

const functionObject = {
    add: function(a, b) { return a + b; },
    subtract: function(a, b) { return a - b; },
    multiply: function(a, b) { return a * b; }
};

// 3. Be passed as arguments to other functions
function executeOperation(operation, a, b) {
    return operation(a, b);
}

const result = executeOperation(functionObject.add, 5, 3);
console.log(result); // 8

// 4. Be returned from other functions
function createMultiplier(factor) {
    return function(number) {
        return number * factor;
    };
}

const double = createMultiplier(2);
const triple = createMultiplier(3);

console.log(double(5)); // 10
console.log(triple(5)); // 15

// 5. Have properties and methods (because they are objects)
function example() {
    console.log("I am a function");
}

example.description = "This is a custom property";
example.execute = function() {
    console.log("I can have methods too!");
};

console.log(example.description); // "This is a custom property"
example.execute(); // "I can have methods too!"

// Functions also have built-in properties
console.log(example.length); // 0 (number of parameters)
console.log(example.name);   // "example"
```

### Practical Applications

**Strategy Pattern:**

```javascript
// Using first-class functions to implement strategy pattern
const paymentStrategies = {
    creditCard: function(amount) {
        console.log(`Processing $${amount} via Credit Card`);
        return { success: true, fee: amount * 0.02 };
    },
    
    paypal: function(amount) {
        console.log(`Processing $${amount} via PayPal`);
        return { success: true, fee: amount * 0.03 };
    },
    
    crypto: function(amount) {
        console.log(`Processing $${amount} via Cryptocurrency`);
        return { success: true, fee: amount * 0.01 };
    }
};

function processPayment(method, amount) {
    const strategy = paymentStrategies[method];
    
    if (!strategy) {
        throw new Error(`Unknown payment method: ${method}`);
    }
    
    return strategy(amount);
}

// Usage
processPayment('creditCard', 100); // Process via credit card
processPayment('paypal', 250);     // Process via PayPal
```

**Command Pattern:**

```javascript
// Using first-class functions for command queue
const commandQueue = [];

function addCommand(command) {
    commandQueue.push(command);
}

function executeCommands() {
    while (commandQueue.length > 0) {
        const command = commandQueue.shift();
        command.execute();
    }
}

// Define commands as functions
addCommand({
    execute: function() {
        console.log("Validating user input...");
    }
});

addCommand({
    execute: function() {
        console.log("Saving to database...");
    }
});

addCommand({
    execute: function() {
        console.log("Sending confirmation email...");
    }
});

executeCommands();
```

---

## 19.3 Higher-Order Functions

A **higher-order function** is a function that either:
1. Takes one or more functions as arguments, or
2. Returns a function as its result

Higher-order functions are fundamental to functional programming and enable powerful abstraction patterns.

### Functions as Arguments

```javascript
// Built-in higher-order functions
const numbers = [1, 2, 3, 4, 5];

// map() - transforms each element
const doubled = numbers.map(function(num) {
    return num * 2;
});
console.log(doubled); // [2, 4, 6, 8, 10]

// filter() - selects elements based on condition
const evens = numbers.filter(function(num) {
    return num % 2 === 0;
});
console.log(evens); // [2, 4]

// reduce() - aggregates values
const sum = numbers.reduce(function(acc, num) {
    return acc + num;
}, 0);
console.log(sum); // 15

// Creating custom higher-order functions
function filterArray(array, predicate) {
    const result = [];
    
    for (let i = 0; i < array.length; i++) {
        if (predicate(array[i], i, array)) {
            result.push(array[i]);
        }
    }
    
    return result;
}

// Usage
const adults = filterArray([15, 22, 18, 12, 35], function(age) {
    return age >= 18;
});
console.log(adults); // [22, 18, 35]
```

### Functions as Return Values

```javascript
// Function factory pattern
function createValidator(min, max) {
    return function(value) {
        return value >= min && value <= max;
    };
}

const isValidAge = createValidator(0, 120);
const isValidPercentage = createValidator(0, 100);
const isValidRating = createValidator(1, 5);

console.log(isValidAge(25));        // true
console.log(isValidAge(150));       // false
console.log(isValidPercentage(75)); // true
console.log(isValidRating(3));      // true

// Partial application
function partial(fn, ...presetArgs) {
    return function(...laterArgs) {
        return fn(...presetArgs, ...laterArgs);
    };
}

function greet(greeting, name) {
    return `${greeting}, ${name}!`;
}

const sayHelloTo = partial(greet, "Hello");
const sayGoodbyeTo = partial(greet, "Goodbye");

console.log(sayHelloTo("Alice"));    // "Hello, Alice!"
console.log(sayGoodbyeTo("Bob"));    // "Goodbye, Bob!"

// Function composition
function compose(...functions) {
    return function(initialValue) {
        return functions.reduceRight(function(value, fn) {
            return fn(value);
        }, initialValue);
    };
}

function addOne(x) { return x + 1; }
function double(x) { return x * 2; }
function square(x) { return x * x; }

const composed = compose(square, double, addOne);
// Execution: square(double(addOne(3)))
// addOne(3) = 4
// double(4) = 8
// square(8) = 64
console.log(composed(3)); // 64
```

### Common Higher-Order Function Patterns

**Memoization:**

```javascript
// Higher-order function that adds caching to any function
function memoize(fn) {
    const cache = new Map();
    
    return function(...args) {
        const key = JSON.stringify(args);
        
        if (cache.has(key)) {
            console.log(`Cache hit for ${key}`);
            return cache.get(key);
        }
        
        console.log(`Computing for ${key}`);
        const result = fn.apply(this, args);
        cache.set(key, result);
        return result;
    };
}

// Expensive function
function fibonacci(n) {
    if (n <= 1) return n;
    return fibonacci(n - 1) + fibonacci(n - 2);
}

// Memoized version
const memoizedFib = memoize(function(n) {
    if (n <= 1) return n;
    return memoizedFib(n - 1) + memoizedFib(n - 2);
});

console.log(memoizedFib(35)); // Fast after first call
console.log(memoizedFib(35)); // Instant (from cache)
```

**Throttling and Debouncing:**

```javascript
// Throttle: ensures function is called at most once per specified time period
function throttle(fn, limit) {
    let inThrottle = false;
    
    return function(...args) {
        if (!inThrottle) {
            fn.apply(this, args);
            inThrottle = true;
            setTimeout(() => inThrottle = false, limit);
        }
    };
}

// Debounce: delays function execution until after specified time of inactivity
function debounce(fn, delay) {
    let timeoutId;
    
    return function(...args) {
        clearTimeout(timeoutId);
        timeoutId = setTimeout(() => fn.apply(this, args), delay);
    };
}

// Usage examples
const handleScroll = throttle(function() {
    console.log("Scroll position:", window.scrollY);
}, 200);

const handleSearch = debounce(function(query) {
    console.log("Searching for:", query);
    // API call would go here
}, 500);

window.addEventListener('scroll', handleScroll);
searchInput.addEventListener('input', (e) => handleSearch(e.target.value));
```

---

## 19.4 Callback Functions

A **callback function** is a function passed as an argument to another function, which is then invoked inside the outer function to complete some kind of routine or action. Callbacks are fundamental to asynchronous programming in JavaScript.

### Synchronous Callbacks

```javascript
// Synchronous callback: executed immediately
function processData(data, callback) {
    console.log("Processing started...");
    const result = data.map(callback);
    console.log("Processing completed!");
    return result;
}

const numbers = [1, 2, 3, 4, 5];
const squared = processData(numbers, function(num) {
    return num * num;
});

console.log(squared); // [1, 4, 9, 16, 25]

// Array method callbacks (synchronous)
const users = [
    { name: "Alice", age: 25 },
    { name: "Bob", age: 30 },
    { name: "Charlie", age: 35 }
];

// forEach with callback
users.forEach(function(user, index) {
    console.log(`${index + 1}. ${user.name} is ${user.age} years old`);
});

// sort with callback
const sorted = users.sort(function(a, b) {
    return a.age - b.age;
});
```

### Asynchronous Callbacks

```javascript
// Simulating async operation with setTimeout
function fetchData(url, callback) {
    console.log(`Fetching data from ${url}...`);
    
    setTimeout(function() {
        const data = { id: 1, name: "Product", price: 99.99 };
        console.log("Data received!");
        callback(data);
    }, 1000);
}

// Usage
fetchData("https://api.example.com/product/1", function(product) {
    console.log(`Product: ${product.name}, Price: $${product.price}`);
});

console.log("This executes before the callback");

// Output order:
// 1. "Fetching data from..."
// 2. "This executes before the callback"
// 3. "Data received!"
// 4. "Product: Product, Price: $99.99"

// Real-world example: Event listeners
document.getElementById('submitBtn').addEventListener('click', function(event) {
    event.preventDefault();
    console.log("Form submitted!");
    // Handle form submission
});
```

### Callback Hell and Solutions

```javascript
// CALLBACK HELL (Pyramid of Doom)
getUserData(userId, function(user) {
    getOrders(user.id, function(orders) {
        getOrderDetails(orders[0].id, function(details) {
            getProductInfo(details.productId, function(product) {
                getInventoryStatus(product.id, function(inventory) {
                    console.log("Final result:", inventory);
                    // Deeply nested, hard to read, hard to maintain
                });
            });
        });
    });
});

// SOLUTION 1: Named functions
function handleInventory(inventory) {
    console.log("Final result:", inventory);
}

function handleProduct(product) {
    getInventoryStatus(product.id, handleInventory);
}

function handleDetails(details) {
    getProductInfo(details.productId, handleProduct);
}

function handleOrders(orders) {
    getOrderDetails(orders[0].id, handleDetails);
}

function handleUser(user) {
    getOrders(user.id, handleOrders);
}

getUserData(userId, handleUser);

// SOLUTION 2: Modularization
function getUserDataAsync(userId) {
    return new Promise(function(resolve) {
        getUserData(userId, resolve);
    });
}

// Using Promises (modern approach)
getUserDataAsync(userId)
    .then(user => getOrdersAsync(user.id))
    .then(orders => getOrderDetailsAsync(orders[0].id))
    .then(details => getProductInfoAsync(details.productId))
    .then(product => getInventoryStatusAsync(product.id))
    .then(inventory => console.log("Final result:", inventory))
    .catch(error => console.error("Error:", error));
```

### Error-First Callback Pattern (Node.js Style)

```javascript
// Industry standard for Node.js callbacks
function readFileAsync(filePath, callback) {
    try {
        // Simulating file read
        setTimeout(function() {
            if (filePath.endsWith('.txt')) {
                const data = "File contents here...";
                callback(null, data); // First argument: error (null if success)
            } else {
                callback(new Error("Invalid file type"), null);
            }
        }, 100);
    } catch (err) {
        callback(err, null);
    }
}

// Usage
readFileAsync('document.txt', function(error, data) {
    if (error) {
        console.error("Failed to read:", error.message);
        return;
    }
    console.log("File contents:", data);
});
```

---

## 19.5 Closures

A **closure** is the combination of a function bundled together (enclosed) with references to its surrounding state (the **lexical environment**). In JavaScript, closures are created every time a function is created, at function creation time.

Closures are one of JavaScript's most powerful and often misunderstood features. They allow functions to "remember" the environment in which they were created.

### Understanding Lexical Scope

```javascript
// Lexical scope means functions are executed using the scope chain 
// that was in effect when they were defined, not when they are executed

const globalVar = "I am global";

function outerFunction() {
    const outerVar = "I am outer";
    
    function innerFunction() {
        const innerVar = "I am inner";
        
        // Can access: innerVar, outerVar, globalVar
        console.log(innerVar);  // "I am inner"
        console.log(outerVar);  // "I am outer"
        console.log(globalVar); // "I am global"
    }
    
    innerFunction();
}

outerFunction();
```

### Creating Closures

```javascript
function makeCounter() {
    // This variable is enclosed in the closure
    let count = 0;
    
    // The returned function forms a closure
    return function() {
        count += 1;
        return count;
    };
}

const counter1 = makeCounter();
const counter2 = makeCounter();

// Each counter has its own independent closure
console.log(counter1()); // 1
console.log(counter1()); // 2
console.log(counter1()); // 3

console.log(counter2()); // 1 (independent)
console.log(counter2()); // 2

// The count variable is private and cannot be accessed directly
// console.log(counter1.count); // undefined
```

### Practical Closure Patterns

**Data Privacy (Private Variables):**

```javascript
function createBankAccount(initialBalance) {
    // Private variable - not accessible from outside
    let balance = initialBalance;
    
    // Transaction history (also private)
    const transactions = [];
    
    return {
        deposit: function(amount) {
            if (amount <= 0) {
                throw new Error("Amount must be positive");
            }
            balance += amount;
            transactions.push({ type: 'deposit', amount, date: new Date() });
            return balance;
        },
        
        withdraw: function(amount) {
            if (amount > balance) {
                throw new Error("Insufficient funds");
            }
            balance -= amount;
            transactions.push({ type: 'withdrawal', amount, date: new Date() });
            return balance;
        },
        
        getBalance: function() {
            return balance;
        },
        
        getTransactionHistory: function() {
            // Return a copy to prevent external modification
            return [...transactions];
        }
    };
}

const account = createBankAccount(1000);
account.deposit(500);
account.withdraw(200);
console.log(account.getBalance()); // 1300

// Cannot access balance directly
console.log(account.balance); // undefined
```

**Function Factories:**

```javascript
function makeMultiplier(multiplier) {
    // multiplier is captured in the closure
    return function(number) {
        return number * multiplier;
    };
}

const double = makeMultiplier(2);
const triple = makeMultiplier(3);
const quadruple = makeMultiplier(4);

console.log(double(5));    // 10
console.log(triple(5));    // 15
console.log(quadruple(5)); // 20

// Each function remembers its own multiplier value
```

**Configuration/Partial Application:**

```javascript
function createGreeter(greeting) {
    return function(name) {
        return `${greeting}, ${name}!`;
    };
}

const sayHello = createGreeter("Hello");
const sayHi = createGreeter("Hi");
const sayGoodMorning = createGreeter("Good morning");

console.log(sayHello("Alice"));      // "Hello, Alice!"
console.log(sayHi("Bob"));           // "Hi, Bob!"
console.log(sayGoodMorning("Carol")); // "Good morning, Carol!"
```

**Maintaining State in Async Operations:**

```javascript
function setupButton(buttonId, buttonName) {
    const button = document.getElementById(buttonId);
    
    // buttonName is preserved in the closure even though
    // the event listener executes later
    button.addEventListener('click', function() {
        console.log(`Button "${buttonName}" was clicked!`);
    });
}

setupButton('btn1', 'Submit');
setupButton('btn2', 'Cancel');

// Even if setupButton has finished executing, the event listeners
// still have access to their respective buttonName values
```

### Common Closure Pitfalls

**The Loop Problem:**

```javascript
// PROBLEM: All buttons log the same number (5)
for (var i = 0; i < 5; i++) {
    setTimeout(function() {
        console.log("Index:", i); // Always logs 5
    }, 100);
}

// SOLUTION 1: Use let (block-scoped)
for (let i = 0; i < 5; i++) {
    setTimeout(function() {
        console.log("Index:", i); // 0, 1, 2, 3, 4
    }, 100);
}

// SOLUTION 2: IIFE to create new scope
for (var i = 0; i < 5; i++) {
    (function(capturedIndex) {
        setTimeout(function() {
            console.log("Index:", capturedIndex); // 0, 1, 2, 3, 4
        }, 100);
    })(i);
}

// SOLUTION 3: Pass as argument to setTimeout
for (var i = 0; i < 5; i++) {
    setTimeout(function(index) {
        console.log("Index:", index);
    }, 100, i); // Third argument passed to callback
}
```

**Memory Considerations:**

```javascript
// Closures keep references to outer variables, preventing garbage collection
function heavyDataProcessor() {
    const hugeArray = new Array(1000000).fill('data'); // Large data
    
    return function(index) {
        // This closure keeps hugeArray in memory
        return hugeArray[index];
    };
}

const getData = heavyDataProcessor();
// hugeArray remains in memory because getData references it

// To release memory:
getData = null; // Now hugeArray can be garbage collected
```

---

## 19.6 Immediately Invoked Function Expressions (IIFE)

An **IIFE** (pronounced "iffy") is a function that is executed immediately after it is defined. It is a design pattern used to create a new scope and avoid polluting the global namespace.

### IIFE Syntax

```javascript
// Basic IIFE structure
(function() {
    // Code here runs immediately
    console.log("IIFE executed!");
})();

// Alternative syntax (less common)
(function() {
    console.log("IIFE executed!");
}());

// Named IIFE (useful for debugging)
(function initApplication() {
    console.log("Application initialized!");
})();

// IIFE with parameters
(function(name, version) {
    console.log(`Starting ${name} v${version}`);
})("MyApp", "1.0.0");
```

### Why Use IIFEs?

**1. Encapsulation and Privacy:**

```javascript
// Without IIFE - pollutes global scope
var counter = 0; // Global variable (bad!)
function increment() {
    counter++;
}

// With IIFE - encapsulation
const counterModule = (function() {
    let count = 0; // Private variable
    
    return {
        increment: function() {
            count++;
            return count;
        },
        decrement: function() {
            count--;
            return count;
        },
        getCount: function() {
            return count;
        }
    };
})();

console.log(counterModule.getCount()); // 0
counterModule.increment();
console.log(counterModule.getCount()); // 1
// console.log(count); // ReferenceError: count is not defined
```

**2. Avoiding Global Scope Pollution:**

```javascript
// Old pattern to avoid global variables
(function(global) {
    // All variables here are local to this IIFE
    const privateUtil = {
        helper: function() {
            console.log("Helper function");
        }
    };
    
    function publicAPI() {
        privateUtil.helper();
    }
    
    // Expose only what needs to be global
    global.MyLibrary = {
        doSomething: publicAPI
    };
    
})(window); // Pass window as parameter to ensure correct reference

// Usage
MyLibrary.doSomething();
// MyLibrary.privateUtil.helper(); // Error - private!
```

**3. Creating Fresh Scope in Loops (Pre-ES6):**

```javascript
// Before let/const, IIFEs were used to capture loop variables
var buttons = document.querySelectorAll('.btn');

for (var i = 0; i < buttons.length; i++) {
    (function(capturedIndex) {
        buttons[i].addEventListener('click', function() {
            console.log('Button ' + capturedIndex + ' clicked');
        });
    })(i);
}
```

**4. Module Pattern:**

```javascript
const ShoppingCart = (function() {
    // Private
    const items = [];
    let total = 0;
    
    function calculateTotal() {
        total = items.reduce((sum, item) => sum + (item.price * item.quantity), 0);
    }
    
    // Public API
    return {
        addItem: function(name, price, quantity = 1) {
            items.push({ name, price, quantity });
            calculateTotal();
        },
        
        removeItem: function(name) {
            const index = items.findIndex(item => item.name === name);
            if (index > -1) {
                items.splice(index, 1);
                calculateTotal();
            }
        },
        
        getTotal: function() {
            return total;
        },
        
        getItemCount: function() {
            return items.length;
        },
        
        clear: function() {
            items.length = 0;
            total = 0;
        }
    };
})();

ShoppingCart.addItem("Laptop", 999.99, 1);
ShoppingCart.addItem("Mouse", 29.99, 2);
console.log(ShoppingCart.getTotal()); // 1059.97
```

**5. Configuration and Setup:**

```javascript
const AppConfig = (function() {
    const defaults = {
        apiUrl: 'https://api.example.com',
        timeout: 5000,
        retries: 3,
        debug: false
    };
    
    // Check for environment overrides
    const environment = typeof window !== 'undefined' ? 'browser' : 'node';
    
    if (environment === 'browser' && window.location.hostname === 'localhost') {
        defaults.debug = true;
        defaults.apiUrl = 'http://localhost:3000';
    }
    
    return defaults;
})();

console.log(AppConfig.apiUrl); // Uses correct URL based on environment
```

### Arrow Function IIFEs

With ES6, you can use arrow functions in IIFEs:

```javascript
// Arrow function IIFE
(() => {
    console.log("Arrow IIFE!");
})();

// With parameters
((name) => {
    console.log(`Hello, ${name}!`);
})("World");

// Returning values
const config = (() => {
    const settings = {
        theme: 'dark',
        language: 'en'
    };
    
    return {
        get: (key) => settings[key],
        set: (key, value) => { settings[key] = value; }
    };
})();
```

### Modern Alternatives to IIFEs

With ES6 modules and block-scoped variables, IIFEs are less necessary, but still useful:

```javascript
// ES6 Block Scope (alternative to simple IIFEs)
{
    const privateVar = "secret";
    let count = 0;
    
    // These are block-scoped, not accessible outside
    console.log(privateVar);
}

// console.log(privateVar); // ReferenceError

// ES6 Modules (alternative to module pattern IIFEs)
// module.js
const privateData = "hidden";
export function publicFunction() {
    return privateData;
}

// main.js
import { publicFunction } from './module.js';
```

---

## 19.7 Pure Functions

A **pure function** is a function that:
1. Given the same inputs, always returns the same output
2. Has no side effects (does not modify external state)

Pure functions are the foundation of functional programming and make code more predictable, testable, and maintainable.

### Characteristics of Pure Functions

```javascript
// PURE FUNCTION
function add(a, b) {
    return a + b;
}

// Always returns same output for same input
console.log(add(2, 3)); // 5
console.log(add(2, 3)); // 5
console.log(add(2, 3)); // 5

// No side effects - doesn't modify anything outside itself

// IMPURE FUNCTION (has side effects)
let total = 0;
function addToTotal(amount) {
    total += amount; // Side effect: modifies external variable
    return total;
}

// Returns different values for same input
console.log(addToTotal(5)); // 5
console.log(addToTotal(5)); // 10 (different result!)

// IMPURE FUNCTION (depends on external state)
function getDiscountedPrice(price) {
    return price * (1 - globalDiscountRate); // Depends on external variable
}

// If globalDiscountRate changes, output changes even with same input
```

### Side Effects Examples

```javascript
// SIDE EFFECTS TO AVOID IN PURE FUNCTIONS:

// 1. Modifying external variables
let counter = 0;
function increment() {
    counter++; // Side effect!
}

// 2. Modifying input arguments (mutation)
function addItem(array, item) {
    array.push(item); // Side effect: mutates input!
    return array;
}

// 3. I/O operations
function logMessage(msg) {
    console.log(msg); // Side effect: I/O operation
}

// 4. DOM manipulation
function updateTitle(text) {
    document.title = text; // Side effect: DOM mutation
}

// 5. API calls
function fetchUser(id) {
    return fetch(`/api/users/${id}`); // Side effect: network request
}

// 6. Reading external state (if it can change)
function getTaxRate() {
    return currentTaxRate; // Impure if currentTaxRate can change
}
```

### Refactoring to Pure Functions

```javascript
// BEFORE: Impure shopping cart
const cart = [];

function addToCart(item) {
    cart.push(item); // Mutates external cart
    console.log(`Added ${item.name}`); // I/O side effect
    updateCartUI(); // DOM side effect
}

// AFTER: Pure version
function addToCartPure(cart, item) {
    // Returns new array instead of mutating
    return [...cart, item];
}

function createCartActions(cart, setCart) {
    return {
        addItem: (item) => {
            const newCart = addToCartPure(cart, item);
            setCart(newCart); // State update handled separately
            return newCart;
        },
        
        removeItem: (itemId) => {
            return cart.filter(item => item.id !== itemId);
        },
        
        updateQuantity: (itemId, quantity) => {
            return cart.map(item => 
                item.id === itemId 
                    ? { ...item, quantity } 
                    : item
            );
        },
        
        getTotal: () => {
            return cart.reduce((sum, item) => sum + (item.price * item.quantity), 0);
        }
    };
}
```

### Benefits of Pure Functions

```javascript
// 1. PREDICTABILITY: Easy to reason about
function calculateTotal(items, taxRate) {
    const subtotal = items.reduce((sum, item) => sum + item.price * item.quantity, 0);
    return subtotal * (1 + taxRate);
}

// 2. TESTABILITY: No setup required
// Test:
const testItems = [{ price: 10, quantity: 2 }, { price: 5, quantity: 1 }];
const result = calculateTotal(testItems, 0.1);
console.assert(result === 27.5, "Calculation should be correct");

// 3. CACHABILITY (Memoization)
const memoizedCalculate = memoize(calculateTotal);
// Same inputs = instant cached result

// 4. CONCURRENCY: Safe to run in parallel
// No shared state = no race conditions

// 5. TIME TRAVEL DEBUGGING
// Can replay state changes because each state is immutable
```

### Pure Function Utilities

```javascript
// Pure array methods (return new arrays)
const numbers = [1, 2, 3];

// Pure: map, filter, reduce, concat, slice, spread
const doubled = numbers.map(n => n * 2); // [2, 4, 6]
const evens = numbers.filter(n => n % 2 === 0); // [2]
const sum = numbers.reduce((a, b) => a + b, 0); // 6

// Impure: push, pop, shift, unshift, splice, sort, reverse
numbers.push(4); // Mutates original!

// Pure alternatives to mutating methods
function purePush(array, item) {
    return [...array, item];
}

function purePop(array) {
    return array.slice(0, -1);
}

function pureSort(array, compareFn) {
    return [...array].sort(compareFn);
}

// Usage
const sorted = pureSort([3, 1, 4, 1, 5], (a, b) => a - b);
console.log(sorted); // [1, 1, 3, 4, 5]
```

### When to Use Impure Functions

While pure functions are preferred, impure functions are necessary for:

```javascript
// 1. I/O Operations (required for any useful program)
function saveToDatabase(data) {
    // Impure but necessary
    return db.collection('users').insert(data);
}

// 2. Generating Random Numbers
function rollDice() {
    return Math.floor(Math.random() * 6) + 1; // Impure (non-deterministic)
}

// 3. Getting Current Time
function getTimestamp() {
    return Date.now(); // Impure (different each call)
}

// 4. DOM Updates (UI requires side effects)
function renderComponent(component) {
    document.getElementById('root').innerHTML = component.html;
}

// STRATEGY: Isolate impure functions, keep business logic pure
function generateReport(data) {
    // Pure: data transformation
    const processedData = data.map(transformRow).filter(isValid);
    const summary = calculateSummary(processedData);
    
    // Impure: I/O (isolated at the edges)
    const timestamp = getTimestamp();
    saveReport(summary, timestamp);
    
    return summary;
}
```

---

## 19.8 Recursion

**Recursion** is a programming technique where a function calls itself to solve a problem by breaking it down into smaller, identical subproblems. A recursive function must have a **base case** (stopping condition) and a **recursive case** (where it calls itself).

### Basic Recursion Structure

```javascript
function recursiveFunction(parameters) {
    // BASE CASE: Stop condition
    if (/* base condition */) {
        return simpleValue;
    }
    
    // RECURSIVE CASE: Call itself with modified parameters
    return recursiveFunction(modifiedParameters);
}

// Example: Countdown
function countdown(n) {
    // Base case
    if (n <= 0) {
        console.log("Blast off!");
        return;
    }
    
    // Recursive case
    console.log(n);
    countdown(n - 1); // Call itself with n-1
}

countdown(5);
// Output: 5, 4, 3, 2, 1, Blast off!
```

### Classic Recursion Examples

**Factorial:**

```javascript
// Mathematical: n! = n × (n-1) × (n-2) × ... × 1
// 5! = 5 × 4 × 3 × 2 × 1 = 120

// Iterative approach
function factorialIterative(n) {
    let result = 1;
    for (let i = n; i > 1; i--) {
        result *= i;
    }
    return result;
}

// Recursive approach
function factorial(n) {
    // Base case
    if (n <= 1) return 1;
    
    // Recursive case
    return n * factorial(n - 1);
}

// Execution trace for factorial(5):
// factorial(5) = 5 * factorial(4)
//              = 5 * (4 * factorial(3))
//              = 5 * (4 * (3 * factorial(2)))
//              = 5 * (4 * (3 * (2 * factorial(1))))
//              = 5 * (4 * (3 * (2 * 1)))
//              = 120

console.log(factorial(5)); // 120
```

**Fibonacci Sequence:**

```javascript
// Sequence: 0, 1, 1, 2, 3, 5, 8, 13, 21...
// fib(n) = fib(n-1) + fib(n-2)

// Naive recursive (inefficient - exponential time O(2^n))
function fibonacci(n) {
    if (n <= 1) return n;
    return fibonacci(n - 1) + fibonacci(n - 2);
}

// Optimized with memoization
function fibonacciMemo(n, memo = {}) {
    if (n in memo) return memo[n];
    if (n <= 1) return n;
    
    memo[n] = fibonacciMemo(n - 1, memo) + fibonacciMemo(n - 2, memo);
    return memo[n];
}

// Tail recursive (ES6+ engines may optimize)
function fibonacciTail(n, a = 0, b = 1) {
    if (n === 0) return a;
    if (n === 1) return b;
    return fibonacciTail(n - 1, b, a + b);
}

console.log(fibonacciMemo(50)); // 12586269025 (instant)
console.log(fibonacciTail(50)); // 12586269025 (instant)
```

### Recursion on Data Structures

**Traversing Nested Arrays:**

```javascript
function flattenArray(arr) {
    let result = [];
    
    for (let item of arr) {
        if (Array.isArray(item)) {
            // Recursive case: flatten the nested array and concat
            result = result.concat(flattenArray(item));
        } else {
            // Base case: push the non-array item
            result.push(item);
        }
    }
    
    return result;
}

const nested = [1, [2, 3], [[4, 5], 6], [[[7]]]];
console.log(flattenArray(nested)); // [1, 2, 3, 4, 5, 6, 7]
```

**Tree Traversal:**

```javascript
const fileSystem = {
    name: "root",
    type: "folder",
    children: [
        {
            name: "documents",
            type: "folder",
            children: [
                { name: "resume.pdf", type: "file", size: 120 },
                { name: "coverletter.docx", type: "file", size: 45 }
            ]
        },
        {
            name: "photos",
            type: "folder",
            children: [
                { name: "vacation.jpg", type: "file", size: 2048 },
                { name: "family.png", type: "file", size: 1536 }
            ]
        }
    ]
};

// Calculate total size recursively
function calculateTotalSize(node) {
    // Base case: file
    if (node.type === "file") {
        return node.size;
    }
    
    // Base case: empty folder
    if (!node.children || node.children.length === 0) {
        return 0;
    }
    
    // Recursive case: sum sizes of all children
    return node.children.reduce((total, child) => {
        return total + calculateTotalSize(child);
    }, 0);
}

console.log(calculateTotalSize(fileSystem)); // 3749

// Find all files recursively
function findAllFiles(node, files = []) {
    if (node.type === "file") {
        files.push(node.name);
    } else if (node.children) {
        node.children.forEach(child => findAllFiles(child, files));
    }
    return files;
}

console.log(findAllFiles(fileSystem)); 
// ["resume.pdf", "coverletter.docx", "vacation.jpg", "family.png"]
```

**Deep Clone:**

```javascript
function deepClone(obj) {
    // Base case: null or not an object
    if (obj === null || typeof obj !== "object") {
        return obj;
    }
    
    // Handle Date
    if (obj instanceof Date) {
        return new Date(obj.getTime());
    }
    
    // Handle Array
    if (Array.isArray(obj)) {
        return obj.map(item => deepClone(item));
    }
    
    // Handle Object
    const cloned = {};
    for (let key in obj) {
        if (obj.hasOwnProperty(key)) {
            cloned[key] = deepClone(obj[key]);
        }
    }
    
    return cloned;
}

const original = {
    name: "John",
    address: {
        city: "New York",
        coordinates: [40.7128, 74.0060]
    },
    hobbies: ["reading", "gaming"]
};

const cloned = deepClone(original);
cloned.address.city = "Boston";

console.log(original.address.city); // "New York" (unchanged)
console.log(cloned.address.city);   // "Boston"
```

### Recursion vs Iteration

```javascript
// Most recursive functions can be written iteratively
// Trade-offs to consider:

// RECURSION
// Pros:
// - Often more readable for tree/graph traversal
// - Declarative, mathematical elegance
// - Natural fit for divide-and-conquer algorithms
// Cons:
// - Risk of stack overflow for deep recursion
// - Function call overhead
// - May be harder to debug

// ITERATION
// Pros:
// - Better performance (no call stack overhead)
// - No risk of stack overflow
// - Often more memory efficient
// Cons:
// - Can be less readable for complex structures
// - Requires manual state management

// Example: Sum of array
// Recursive
function sumRecursive(arr) {
    if (arr.length === 0) return 0;
    return arr[0] + sumRecursive(arr.slice(1));
}

// Iterative
function sumIterative(arr) {
    let sum = 0;
    for (let num of arr) {
        sum += num;
    }
    return sum;
}

// For large arrays, iterative is safer
const largeArray = new Array(10000).fill(1);
console.log(sumIterative(largeArray)); // 10000
// sumRecursive(largeArray); // Stack overflow!
```

### Tail Call Optimization

```javascript
// Tail recursion: recursive call is the last operation
// Some JavaScript engines optimize this to prevent stack overflow

// NOT tail recursive (must multiply after recursive call returns)
function factorialNotTail(n) {
    if (n <= 1) return 1;
    return n * factorialNotTail(n - 1); // Multiplication happens after
}

// Tail recursive (accumulator pattern)
function factorialTail(n, accumulator = 1) {
    if (n <= 1) return accumulator;
    return factorialTail(n - 1, n * accumulator); // Last operation is the call
}

// Note: Tail call optimization is only guaranteed in strict mode
// and not all JavaScript engines implement it fully
```

### When to Use Recursion

**Use recursion when:**
- Working with tree structures (DOM, file system, organizational charts)
- Implementing divide-and-conquer algorithms (merge sort, quick sort)
- Problems naturally defined recursively (factorial, fibonacci)
- Backtracking algorithms (maze solving, sudoku)
- When code clarity is more important than micro-optimization

**Avoid recursion when:**
- The recursion depth could exceed the call stack limit (usually ~10,000)
- Performance is critical and iteration is straightforward
- The problem is simple linear iteration
- Working with very large datasets

---

## Chapter Summary

In this chapter, you explored the advanced capabilities of JavaScript functions:

1. **Function Declarations vs Expressions**: Declarations are hoisted and have different scoping rules than expressions. Expressions offer more flexibility and are generally preferred for modern development.

2. **First-Class Functions**: JavaScript treats functions as values, allowing them to be assigned to variables, stored in data structures, passed as arguments, and returned from other functions. This enables powerful abstraction patterns.

3. **Higher-Order Functions**: Functions that operate on other functions (taking them as arguments or returning them) enable functional programming patterns like map/filter/reduce, memoization, throttling, and composition.

4. **Callback Functions**: Functions passed as arguments to be executed later are fundamental to asynchronous programming. Understanding callback patterns and avoiding "callback hell" is essential for clean async code.

5. **Closures**: Functions that remember their lexical scope even when executed elsewhere enable data privacy, state encapsulation, and the module pattern. They are one of JavaScript's most powerful features.

6. **IIFEs**: Immediately Invoked Function Expressions create private scopes, prevent global namespace pollution, and implement the module pattern. While less common with ES6 modules, they remain useful for specific encapsulation needs.

7. **Pure Functions**: Functions without side effects that return consistent outputs for given inputs make code predictable, testable, and maintainable. They form the foundation of functional programming.

8. **Recursion**: Functions that call themselves solve problems by breaking them into smaller subproblems. Essential for tree traversal and divide-and-conquer algorithms, but requires careful handling of base cases and stack limits.

### Key Takeaways

- Prefer function expressions with `const` for most use cases
- Leverage closures for data privacy and state management
- Use higher-order functions to abstract common patterns
- Keep functions pure when possible; isolate side effects
- Understand recursion for tree/graph operations but be mindful of stack limits
- Modern JavaScript (ES6+) reduces the need for IIFEs but they remain valid patterns

### Practice Exercises

1. Create a function that returns a counter function using closures. Each call should increment and return the count, but the count should not be accessible from outside.

2. Implement `map`, `filter`, and `reduce` using only `for` loops (recreating these higher-order functions).

3. Write a memoization function that can wrap any function and cache its results based on arguments.

4. Create a recursive function to deeply freeze an object (make all nested objects immutable).

5. Refactor a nested callback structure into a flat chain using Promises or async/await.

6. Implement a debounce function using closures to limit how often a function can fire.

---

## Coming Up Next

**Chapter 20: The DOM — Document Object Model**

In the next chapter, you will learn how JavaScript interacts with HTML documents through the DOM API. This bridges your JavaScript knowledge with the web page structure:

- Understanding the DOM tree structure and node relationships
- Selecting elements using various methods (getElementById, querySelector, etc.)
- Traversing the DOM (parent, child, sibling navigation)
- Manipulating elements (creating, modifying, removing)
- Working with element properties, attributes, and styles
- Understanding DOM performance and reflows/repaints

The DOM is the interface between your JavaScript code and the visual web page, making this chapter essential for creating interactive web applications.

---